In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')
import glob
!{sys.executable} -m pip install numpy pandas matplotlib scikit-learn pyarrow seaborn imblearn xgboost| grep -v 'already satisfied'

#

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
df.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage,Status
0,0,999,D-penicillamine,21532,M,N,N,N,N,2.3,316.0,3.35,172.0,1601.0,179.80,63.0,394.0,9.7,3.0,D
1,1,2574,Placebo,19237,F,N,N,N,N,0.9,364.0,3.54,63.0,1440.0,134.85,88.0,361.0,11.0,3.0,C
2,2,3428,Placebo,13727,F,N,Y,Y,Y,3.3,299.0,3.55,131.0,1029.0,119.35,50.0,199.0,11.7,4.0,D
3,3,2576,Placebo,18460,F,N,N,N,N,0.6,256.0,3.50,58.0,1653.0,71.30,96.0,269.0,10.7,3.0,C
4,4,788,Placebo,16658,F,N,Y,N,N,1.1,346.0,3.65,63.0,1181.0,125.55,96.0,298.0,10.6,4.0,C


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7905 entries, 0 to 7904
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             7905 non-null   int64  
 1   N_Days         7905 non-null   int64  
 2   Drug           7905 non-null   object 
 3   Age            7905 non-null   int64  
 4   Sex            7905 non-null   object 
 5   Ascites        7905 non-null   object 
 6   Hepatomegaly   7905 non-null   object 
 7   Spiders        7905 non-null   object 
 8   Edema          7905 non-null   object 
 9   Bilirubin      7905 non-null   float64
 10  Cholesterol    7905 non-null   float64
 11  Albumin        7905 non-null   float64
 12  Copper         7905 non-null   float64
 13  Alk_Phos       7905 non-null   float64
 14  SGOT           7905 non-null   float64
 15  Tryglicerides  7905 non-null   float64
 16  Platelets      7905 non-null   float64
 17  Prothrombin    7905 non-null   float64
 18  Stage   

In [5]:
# create train and validation sets that are divided equally

In [6]:
from sklearn.model_selection import train_test_split

train, validation = train_test_split(df, stratify=df["Status"], test_size=0.2)

In [7]:
numeric_values = ["N_Days","Age","Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets","Prothrombin","Stage"]

In [8]:
categorical_values = ["Drug","Sex","Ascites","Hepatomegaly","Spiders","Edema"]
categorical_bin_values = ["Drug","Sex","Ascites","Hepatomegaly","Spiders"]
categorical_oh_values = ["Edema"]

In [9]:
features_to_scale = ["N_Days","Age","Bilirubin","Cholesterol","Albumin","Copper","Alk_Phos","SGOT","Tryglicerides","Platelets","Prothrombin","Stage"]

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder

# numeric pipeline
num_pipeline = Pipeline([('func_transformer', FunctionTransformer(np.log1p)), 
                         ('minmax_scaler', MinMaxScaler()), ])




In [11]:
from sklearn.base import BaseEstimator, TransformerMixin 
class BinaryEncoder (BaseEstimator, TransformerMixin):
    def __init__ (self, features):
        self.features = features
    def fit(self, X, y=None): 
        return self # do nothing
    def transform(self, X):
        for c in self.features:
            values = X[c].unique()
            X[c].replace(values,(0,1), inplace=True)
        return X
        
        

In [12]:
# full pipeline (numeric & categorical)
full_pipeline = ColumnTransformer([ ("num", num_pipeline, numeric_values), 
                                   ("bin_cat", BinaryEncoder(categorical_bin_values), categorical_bin_values), 
                                   ("cat", OneHotEncoder(sparse_output=False), categorical_oh_values), ]) 
train_prepared = full_pipeline.fit_transform(train)

In [13]:
# transform with pipeline
train_prepared = full_pipeline.fit_transform(train)

In [14]:
pd.DataFrame(train_prepared).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.808203,0.443609,0.122222,0.539166,0.677080,0.925565,0.569465,0.437996,0.331478,0.688599,0.231236,1.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.945695,0.483095,0.066874,0.229255,0.663473,0.460732,0.170167,0.171896,0.453816,0.682151,0.284055,1.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.759818,0.282541,0.385274,0.497931,0.700607,0.556353,0.619828,0.394483,0.436670,0.201573,0.271017,0.756471,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
3,0.867251,0.483095,0.046088,0.464144,0.683838,0.165330,0.338322,0.387808,0.173931,0.810139,0.105411,0.442507,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
4,0.963794,0.611391,0.104808,0.605224,0.632415,0.413941,0.194393,0.338441,0.323464,0.775763,0.162592,0.756471,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0


In [17]:
# Train Model
from sklearn.ensemble import GradientBoostingClassifier
    # Label Encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_en = label_encoder.fit_transform(train["Status"])
model_xgb = GradientBoostingClassifier().fit(train_prepared, labels_en)
# Cross Validation
from sklearn.model_selection import KFold, cross_val_score
xgb_scores = cross_val_score(model_xgb, train_prepared, labels_en, scoring="neg_log_loss", cv=5)


In [19]:
print(-xgb_scores)
print(-xgb_scores.mean())

[0.46494962 0.45188196 0.45094335 0.44876964 0.4503321 ]
0.45337533630378674


In [ ]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels_en = label_encoder.fit_transform(train["status"])
# k-fold (e.g. 5) validation, with log loss, scores need to be un-negated back
# scores = cross_val_score(model, train_prepared, labels_en, scoring="neg_log_loss", cv=5)
